In [1]:
import json, requests
from bs4 import BeautifulSoup
import sys
import time
import urllib.parse as urlparse
from urllib.parse import urlencode

In [2]:
u = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
parameter = {
    'api-key': "46c1eeb270c94547869a041d823b86ac",
    'fq':"section_name:(\"sports\")"
}

In [3]:
def getUrl(u,parameter):
    url_parts = list(urlparse.urlparse(u))
    query = dict(urlparse.parse_qsl(url_parts[4]))
    query.update(parameter)
    url_parts[4] = urlencode(query)
    return urlparse.urlunparse(url_parts)

In [4]:
def tryGetData(url):
    resp = requests.get(url)
    data = json.loads(resp.text)
    try:
        hits = data['response']['meta']['hits']
    except KeyError:
        print(data)
        time.sleep(10)
        data,hits = tryGetData(url)
    return data,hits

In [7]:
def getHtmltxt(docid,jsdata):
    webpage = requests.get(jsdata["response"]["docs"][docid]["web_url"])
    soup = BeautifulSoup(webpage.text, 'html.parser')
    p = soup.find_all('p')
    soup = BeautifulSoup(' '.join(str(i) for i in p), 'html.parser')
    return soup.getText()

# change parameters to acheive data from different categories

In [41]:
url = getUrl(u,parameter)
data,hits = tryGetData(url)
docid = 0
print(f'got {hits} results from this field')
print(url)
time.sleep(1)

got 19086 results from this field
http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=46c1eeb270c94547869a041d823b86ac&fq=news_desk%3A%28%22Science%22%29&page=200


In [40]:
parameter["fq"] = "news_desk:(\"Science\")"

In [42]:
page = 0    
while(hits>=1):
    try:
        ids = len(data["response"]["docs"])
    except KeyError:
        print(data)
        time.sleep(10)
        continue
    for docid in range(ids):
        f= open(f"Science/{hits}.txt","a",encoding='utf-8')
        try:
            text = getHtmltxt(docid,data)
        except:
            hits -=1
            continue
        if(len(text)>50):
            f.write(text)
        hits -= 1
        f.close()
        if(hits<1):
            break
        
    page += 1
    if(page>200):
        break
    parameter['page'] = page
    url = getUrl(u,parameter)
    resp = requests.get(url)
    if(hits<1):
        break
    data = json.loads(resp.text)
    time.sleep(1)
page = 0  
print("over")

over
